In [59]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import pandas as pd
# from brainflow.data_filter import DataFilter, FilterTypes, DetrendOperations, WindowOperations
from sklearn.metrics import f1_score
import torch.optim

In [60]:
class ToTensor(object):
    def __call__(self, sample):
        window, labels = sample['window'], sample['labels']
        return {'window': torch.tensor(window.values).to(torch.float32), 
                'labels': torch.tensor(labels).to(torch.float32)}#.values)}

class AbsoluteValue(object):
    def __call__(self, sample):
        window, labels = sample['window'], sample['labels']
        return {'window': torch.abs(window), 
                'labels': labels}

class MinNormalize(object):
    def __call__(self, sample):
        window, labels = sample['window'], sample['labels']
        for channel in range(len(window)):
            #print(len(window))
            min = torch.min(window[channel])
            window[channel] = window[channel] - min
            window[channel] = window[channel] / torch.max(window[channel])
            window[channel] = torch.nan_to_num(window[channel])
        return {'window': (window.unsqueeze(0)), 
                'labels': labels}

# class FixLabels(object):
#     def __call__(self, sample):
#         window, labels = sample['window'], sample['labels']
#         if labels[0] == 2:
#             labels = labels - 1
#         return {'window': window, 
#                 'labels': labels}

class EEGDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.csv_file = pd.read_csv(csv_file, delimiter='\t', header=None)
        self.transform = transform
    
    def __len__(self):
        return len(self.csv_file)
    
    def __getitem__(self, idx):
        # if idx >= len(self.csv_file) / 625:
        #     return

        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sample_size = 625
        
        # window = self.csv_file.iloc[idx*625:idx*625+625, 1:17].transpose()
        window = self.csv_file.iloc[idx*sample_size:idx*sample_size+sample_size, 5:9].transpose()
        labels = [self.csv_file.iloc[idx*sample_size, 32]]#:idx*625+625, 32] # remove brackets for timestep prediction
        sample = {'window': window, 'labels': labels}

        if self.transform:
            sample = self.transform(sample)
        
        return sample

In [61]:
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

eeg_dataset = EEGDataset(csv_file='./randomRBFull.csv', transform=transforms.Compose([ToTensor(),
                                                                                  AbsoluteValue(),
                                                                                  MinNormalize()]))

# eeg_dataset = EEGDataset(csv_file='./fullData.csv', transform=transforms.Compose([ToTensor(),
#                                                                                   AbsoluteValue()]))

#eeg_dataset = EEGDataset(csv_file='./fullData.csv', transform=transforms.Compose([ToTensor()]))

eeg_dataset = torch.utils.data.Subset(eeg_dataset, range(0, 205))

print(eeg_dataset[0]['window'].shape)

#print(eeg_dataset[0]['window'][0][1])

# for channel in range(len(eeg_dataset[0]['window'][0])):
#     print(torch.min(eeg_dataset[0]['window'][0][channel]))

print("Full max:", torch.max(eeg_dataset[0]['window'][0]))
print("Full min:", torch.min(eeg_dataset[0]['window'][0]))

torch.manual_seed(50) # 80
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(eeg_dataset, [0.6, 0.2, 0.2])

# train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

print(len(train_dataset), len(val_dataset), len(test_dataset))

trainreds = 0
trainblues = 0

for i in range(len(train_dataset)):
    if train_dataset[i]['labels'][0] == 1:
        trainblues += 1
    else:
        trainreds += 1

print("Class imbalance:", trainblues, trainreds)

zerocount = 0
onecount = 0

for i in range(len(val_dataset)):
    if val_dataset[i]['labels'][0] == 1:
        onecount += 1
    else:
        zerocount += 1

print(onecount, zerocount)

zerocount = 0
onecount = 0

for i in range(len(test_dataset)):
    if test_dataset[i]['labels'][0] == 1:
        onecount += 1
    else:
        zerocount += 1

print(onecount, zerocount)

print(train_dataset[7])

No GPU available. Training will run on CPU.
torch.Size([1, 4, 625])
Full max: tensor(1.)
Full min: tensor(0.)
123 41 41
Class imbalance: 56 67
22 19
24 17
{'window': tensor([[[0.8504, 0.3451, 0.9403,  ..., 0.0635, 0.5661, 0.1208],
         [0.8749, 0.3324, 0.9719,  ..., 0.0558, 0.6026, 0.0791],
         [0.8453, 0.3582, 0.9891,  ..., 0.0526, 0.5554, 0.1181],
         [0.8705, 0.3974, 1.0000,  ..., 0.0070, 0.4736, 0.0723]]]), 'labels': tensor([1.])}


In [62]:
print(test_dataset[0]['window'].shape)

torch.Size([1, 4, 625])


In [63]:
num_train = len(train_dataset)
window_tensors = torch.empty(size=(num_train, 1, 4, 625))
label_tensors = torch.empty(size=(num_train, 1))

for i in range(num_train):
    window_tensors[i] = train_dataset[i]['window']
    label_tensors[i] = train_dataset[i]['labels']

torch.save(window_tensors, 'window_tensors.pt')
torch.save(label_tensors, 'label_tensors.pt')

window_tensors = torch.load('window_tensors.pt')
label_tensors = torch.load('label_tensors.pt')

print(window_tensors.shape)
print(label_tensors.shape)

torch.Size([123, 1, 4, 625])
torch.Size([123, 1])


In [64]:
num_val = len(val_dataset)
val_window_tensors = torch.empty(size=(num_val, 1, 4, 625))
val_label_tensors = torch.empty(size=(num_val, 1))

for i in range(num_val):
    val_window_tensors[i] = val_dataset[i]['window']
    val_label_tensors[i] = val_dataset[i]['labels']

torch.save(val_window_tensors, 'val_window_tensors.pt')
torch.save(val_label_tensors, 'val_label_tensors.pt')

val_window_tensors = torch.load('val_window_tensors.pt')
val_label_tensors = torch.load('val_label_tensors.pt')

print(val_window_tensors.shape)
print(val_label_tensors.shape)

torch.Size([41, 1, 4, 625])
torch.Size([41, 1])


In [65]:
# class EEGRedBlueClassifier(nn.Module):
#     def __init__(self):
#         super(EEGRedBlueClassifier, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, (2, 5), stride=(1, 5))
#         self.conv2 = nn.Conv2d(32, 64, (2, 5), stride=(1, 3))
#         self.conv3 = nn.Conv2d(64, 128, (2, 5), stride=(1, 1))
#         self.pool1 = nn.AdaptiveAvgPool1d(1)
#         self.fc1 = nn.Linear(128, 16)
#         self.fc2 = nn.Linear(16, 1)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = F.relu(x)
#         x = self.conv2(x)
#         x = F.relu(x)
#         x = x.squeeze()
#         x = self.conv3(x)
#         x = F.relu(x)
#         x = self.pool1(x)
#         #print("Presqueeze:", x)
#         x = x.squeeze()
#         #print("Postsqueeze", x)
#         x = self.fc1(x)
#         x = F.relu(x)
#         x = self.fc2(x)
#         x = F.sigmoid(x)
#         return x
    
#     # def forward(self, x):
#     #     print(x.shape)
#     #     x = self.conv1(x)
#     #     print(x.shape)
#     #     x = F.relu(x)
#     #     x = x.squeeze()
#     #     x = self.conv2(x)
#     #     print(x.shape)
#     #     x = F.relu(x)
#     #     x = self.pool1(x)
#     #     print(x.shape)
#     #     #print("Presqueeze:", x)
#     #     x = x.squeeze()
#     #     #print("Postsqueeze", x)
#     #     print(x.shape)
#     #     x = self.fc1(x)
#     #     print(x.shape)
#     #     x = F.relu(x)
#     #     print(x.shape)
#     #     x = self.fc2(x)
#     #     print(x.shape)
#     #     x = F.sigmoid(x)
#     #     return x


# model = EEGRedBlueClassifier()
# # model(torch.randn((1250,)).unsqueeze(0))

# optimizer = torch.optim.Adam(model.parameters())
# # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min")
# # classcounts = torch.tensor([trainreds, trainblues])
# # maxclass = torch.max(classcounts)
# # classweights = classcounts/maxclass
# # criterion = nn.BCEWithLogitsLoss(weight=classweights)
# criterion = nn.BCELoss()

In [ ]:
class EEGRedBlueClassifier(nn.Module):
    def __init__(self):
        super(EEGRedBlueClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, (4, 5), stride=(1, 5))
        self.pool1 = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(16, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = x.squeeze()
        x = self.pool1(x)
        #print("Presqueeze:", x)
        x = x.squeeze()
        #print("Postsqueeze", x)
        x = self.fc1(x)
        x = F.sigmoid(x)
        return x
    
    # def forward(self, x):
    #     print(x.shape)
    #     x = self.conv1(x)
    #     print(x.shape)
    #     x = F.relu(x)
    #     x = x.squeeze()
    #     x = self.conv2(x)
    #     print(x.shape)
    #     x = F.relu(x)
    #     x = self.pool1(x)
    #     print(x.shape)
    #     #print("Presqueeze:", x)
    #     x = x.squeeze()
    #     #print("Postsqueeze", x)
    #     print(x.shape)
    #     x = self.fc1(x)
    #     print(x.shape)
    #     x = F.relu(x)
    #     print(x.shape)
    #     x = self.fc2(x)
    #     print(x.shape)
    #     x = F.sigmoid(x)
    #     return x


model = EEGRedBlueClassifier()
# model(torch.randn((1250,)).unsqueeze(0))

optimizer = torch.optim.Adam(model.parameters())
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min")
# classcounts = torch.tensor([trainreds, trainblues])
# maxclass = torch.max(classcounts)
# classweights = classcounts/maxclass
# criterion = nn.BCEWithLogitsLoss(weight=classweights)
criterion = nn.BCELoss()

In [67]:
losses = []
lossi = []
vallosses = []
vallossi = []

In [68]:
minValLoss = 9999

for epoch in tqdm(range(2500)):
    for i in range(len(window_tensors)):
        optimizer.zero_grad()
        model.train()
        y_pred = model(window_tensors[i])
        # print(y_pred)

        # if epoch == 0:
        #     print(y_pred)

        loss = criterion(y_pred, label_tensors[i])
        lossi.append(loss.item())
        loss.backward()
        optimizer.step()

        model.eval()
        for i in range(len(val_window_tensors)):
            val_pred = model(val_window_tensors[i])
            loss = criterion(val_pred, val_label_tensors[i])
            vallossi.append(loss.item())
        
        # if epoch <= 1:
        #     for p in model.parameters():
        #         print(p.grad)
    
    losses.append(np.mean(lossi))
    avgvalloss = np.mean(vallossi)
    if avgvalloss < minValLoss:
        minValLoss = avgvalloss
        torch.save(model.state_dict(), 'bestvalmodel.pt')
    vallosses.append(avgvalloss)
    if epoch % 5 == 0:
        plt.plot(losses, "b")
        plt.plot(vallosses, "g")
        plt.savefig('./valloss.png')
        plt.close()
    lossi = []
    vallossi = []
    

plt.plot(losses, "b")
plt.plot(vallosses, "g")

print(losses)

print(losses[-1])
torch.save(model.state_dict(), 'finaltrainmodel.pt')

  0%|          | 0/2500 [00:00<?, ?it/s]


AttributeError: 'EEGRedBlueClassifier' object has no attribute 'fc1'

In [ ]:
minindex = torch.argmin(torch.tensor(vallosses))
maxindex = torch.argmax(torch.tensor(vallosses))
print(losses[minindex], vallosses[minindex])
print(vallosses[maxindex], vallosses[maxindex] - vallosses[minindex])

vallosses = torch.tensor(vallosses)
vallosses1 = torch.concat((vallosses, torch.tensor([0])))
vallosses2 = torch.concat((torch.tensor([0]), vallosses))
print(vallosses)
print(vallosses2)
change = vallosses1 - vallosses2
change

mask = torch.where(change < -1)
mask = mask[0]
mask = mask[:-1]
vallosses[mask + 1]

0.7015886292224978 tensor(0.6919, dtype=torch.float64)
tensor(46.3415, dtype=torch.float64) tensor(45.6495, dtype=torch.float64)
tensor([ 0.6919,  0.6950,  0.6960,  ..., 46.3415, 46.3415, 46.3415],
       dtype=torch.float64)
tensor([ 0.0000,  0.6919,  0.6950,  ..., 46.3415, 46.3415, 46.3415],
       dtype=torch.float64)


C:\Users\grant\AppData\Local\Temp\ipykernel_28032\1732330198.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  minindex = torch.argmin(torch.tensor(vallosses))
C:\Users\grant\AppData\Local\Temp\ipykernel_28032\1732330198.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  maxindex = torch.argmax(torch.tensor(vallosses))
C:\Users\grant\AppData\Local\Temp\ipykernel_28032\1732330198.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vallosses = torch.tensor(vallosses)


tensor([ 1.4969,  0.8402,  1.3120,  1.8067,  1.2715,  2.1720,  1.0741,  0.9761,
        17.1349], dtype=torch.float64)

In [ ]:
modelWeights = torch.load('./bestvalmodel.pt', weights_only=True)
#modelWeights = torch.load('./finaltrainmodel.pt', weights_only=True)
model = EEGRedBlueClassifier()
model.load_state_dict(modelWeights)
model.eval()
truepreds = 0
falsepreds = 0
truepositives = 0
falsepositives = 0
truenegatives = 0
falsenegatives = 0
preds = []
labels = []

with torch.no_grad():
    for i in range(len(test_dataset)):
        if i < 3:
            print(test_dataset[i]['window'])
        y_pred = model(test_dataset[i]['window'])
        # print(y_pred, test_dataset[i]['labels'])
        pred = (y_pred >= 0.5)
        preds.append(pred)
        label = test_dataset[i]['labels']
        labels.append(label)
        if pred == label:
            truepreds += 1
            if pred == 1:
                truepositives += 1
            else:
                truenegatives += 1
        else:
            falsepreds += 1
            if pred == 1:
                falsepositives += 1
            else:
                falsenegatives += 1

print(truepreds / (truepreds + falsepreds))
print(truepositives, truenegatives, falsepositives, falsenegatives)

f1Score = 2 * truepositives / (2 * truepositives + falsepositives + falsenegatives)
print(f1Score)

f1ScoreSKL = f1_score(labels, preds)
print(f1ScoreSKL)

tensor([[[0.6632, 0.1735, 0.6262,  ..., 0.2996, 0.7880, 0.3973],
         [0.6522, 0.1699, 0.6474,  ..., 0.3195, 0.8326, 0.4185],
         [0.6819, 0.2224, 0.6351,  ..., 0.4058, 0.8553, 0.4930],
         [0.6607, 0.2236, 0.6473,  ..., 0.3449, 0.8077, 0.4461]]])
tensor([[[0.2945, 0.7788, 0.3115,  ..., 0.5502, 0.1111, 0.6282],
         [0.3294, 0.8143, 0.3244,  ..., 0.5050, 0.0453, 0.5855],
         [0.3240, 0.7737, 0.3431,  ..., 0.5445, 0.1493, 0.6312],
         [0.3711, 0.8195, 0.3883,  ..., 0.5060, 0.1070, 0.6036]]])
tensor([[[0.2670, 0.6612, 0.1402,  ..., 0.9310, 0.3027, 0.8677],
         [0.2407, 0.5963, 0.1166,  ..., 0.9585, 0.3629, 0.9098],
         [0.2939, 0.6864, 0.1657,  ..., 0.9295, 0.3387, 0.8772],
         [0.2635, 0.6475, 0.1404,  ..., 0.9460, 0.3967, 0.8899]]])
0.4146341463414634
0 17 0 24
0.0
0.0


In [ ]:
print((eeg_dataset[0]['window']))
#print(torch.min((no_normalize[0]['window'] - torch.min(no_normalize[0]['window']))/(torch.max(no_normalize[0]['window']))))

tensor([[[0.5358, 0.5925, 0.1320,  ..., 0.4419, 0.8587, 0.4527],
         [0.4906, 0.6189, 0.1730,  ..., 0.5227, 0.9043, 0.5071],
         [0.4568, 0.5188, 0.1201,  ..., 0.4603, 0.8533, 0.4747],
         [0.4657, 0.5531, 0.1528,  ..., 0.4480, 0.8364, 0.4332]]])


In [ ]:
# torch.save(model, 'oct7model.pt')